In [2]:
from itertools import repeat

from matplotlib.patches import Arc, Ellipse
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [3]:
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 96

# Constants

In [4]:
figsize = (9, 4)

In [5]:
F = ss.norm.cdf

r = golf_ball_radius = 1.68 / 2  # inches
R = hole_radius = 4.25 / 2       # inches

R /= 12
r /= 12

In [6]:
σ = 1.5
α_min = -3.5 * σ
α_max = 3.5 * σ

x_max = 10
xlim1 = -r / 2
xlim_buffer = .5
xlim2 = x_max + R + xlim_buffer
ylim1, ylim2 = -1, 1
s = 2.7 * (ylim2 - ylim1) / (xlim2 - xlim1)

# Utils

In [7]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation


def make_animation(animator, *args, repeat=False):
    animation = FuncAnimation(
        animator.fig,
        animator,
        list(zip(*args)),
        interval=10,
        repeat=repeat
    )
    plt.close()
    html = animation.to_html5_video()
    return html.replace('controls autoplay', 'onclick="this.play()"')

In [22]:
class Animation:
    def __init__(self, threshold_hook=None, pdf=False, figsize=figsize):
        self.fig, self.ax = plt.subplots(figsize=figsize)
        self.threshold_hook = threshold_hook
        self.pdf = pdf
        self._label_pdf = False

    def __call__(self, args):
        puts, x, *pdf_range = args

        self.ax.clear()

        self._init_green(x)
        
        α_max = np.arcsin((R - r) / x) * 180 / np.pi
        for αi, xi in puts:
            self._show_ball(αi, xi, α_max)

        self._show_angle(max([αi for αi, _ in puts]), x, xi)
        self._show_threshold(αi, x, α_max)
        
        if self.pdf and pdf_range[0] is not None:
            α_lim = pdf_range[0]
            if len(pdf_range) == 1:
                α_ref = α_lim
            else:
                α_ref = pdf_range[1]
            self._show_pdf(α_lim, α_ref, x, α_max)
            self._label_pdf = self._label_pdf or (α_lim != α_min and α_ref > 0)

        self._format_plots(x, label_pdf=self._label_pdf)

    def _init_green(self, x):
        # plot the cup
        self.ax.add_patch(Ellipse((x, 0), R*2/s, R*2, color='k', lw=1))
        
    def _show_threshold(self, α, x, α_max):
        if self.threshold_hook is not None:
            α_lower, α_upper = self.threshold_hook(α, x, α_max)
        else:
            α_lower, α_upper = -α_max, α_max

        self.ax.fill_between([0, x], [0, np.arctan(α_lower / 180 * np.pi) * x], zorder=-1, color='C0')
        self.ax.fill_between([0, x], [0, np.arctan(α_upper / 180 * np.pi) * x], zorder=-1, color='C0')

    def _show_ball(self, α, xi, α_max):
        # plot the ball
        y =  np.arctan(α / 180 * np.pi) * xi
        self.ax.add_patch(
            Ellipse(
                (xi, y),
                r*2/s,
                r*2,
                facecolor='C0' if abs(α) < α_max else 'w',
                edgecolor='k',
                lw=1,
                zorder=10
            )
        )
        # plot the trajectory to ball
        self.ax.plot([0, xi], [0, y], color='C7', ls='--')

    def _show_angle(self, α, x, xi):
        # plot x-axis
        self.ax.hlines(0, xlim1, x, color='C7', ls='--')

        # plot angle between x-axis and put
        self.ax.add_patch(
            Arc(
                (0, 0),
                x/s / 5,
                x / 5,
                theta1=0 if α > 0 else α,
                theta2=α if α > 0 else 0,
            )
        )
        if α > 0:
            self.ax.annotate(r'$\alpha$:= Angle of Shot',
                            (x/6, max([np.arctan(α / 180 * np.pi) * x / 6, .05])),
                            ha='center',
                            rotation=α / s)
        else:
            self.ax.annotate(r'$\alpha$:= Angle of Shot',
                            (x/6, 0.05),
                            ha='center',
                            rotation=0)

    def _show_pdf(self, α_lim, α_ref, x, α_max):
        angle = np.linspace(α_min, α_lim, num=300)
        ix1, ix2 = np.argmin(np.abs(angle - (R - r))), np.argmin(np.abs(angle - (-R + r)))
        probs = ss.norm(loc=0, scale=σ).pdf(angle) * 4.5
        for ix, kwargs in [(((angle > -α_max) & (angle < α_max)), dict(color='C0', edgecolor='C0', zorder=10)),
                           (((angle < -α_max) | (angle > α_max)), dict(color='w', edgecolor='k', alpha=.5))]:
            self.ax.fill_betweenx(
                np.arctan(angle / 180 * np.pi) * x,
                x + xlim_buffer,
                x + xlim_buffer + probs,
                where=ix,
                **kwargs)
        self.ax.hlines(
            np.arctan(α_ref / 180 * np.pi) * x,
            x + xlim_buffer,
            x + xlim_buffer + ss.norm(loc=0, scale=σ).pdf(α_ref) * 4.5,
            color='k',
            lw=1.2,
            zorder=20,
        )

    def _format_plots(self, x, label_pdf):
        self.fig.suptitle(r'Probability Put Success')

        self.ax.set_xlabel('Distance to Hole')
        self.ax.set_xticks([0, x])
        self.ax.set_xticklabels(['0', '$x$'])
        self.ax.set_xlim([xlim1, max([self.ax.get_xlim()[1], xlim2])])

        self.ax.set_yticks([])
        self.ax.set_ylim([ylim1, ylim2])

        self.ax.fill_between([], [], color='C0', label='put success')
        self.ax.fill_between([], [], color='w', edgecolor='k', label='put miss')
        self.ax.legend(loc='upper left')

        self.ax.spines.left.set_visible(False)
        self.ax.spines.top.set_visible(False)
        self.ax.spines.right.set_visible(False)
        
        if label_pdf:
            self.ax.annotate(r'$N(0,\sigma$)', (.88, -0.065), xycoords='axes fraction', ha='center')
        
        self.fig.tight_layout()

# Threshold

In [8]:
def threshold_hook(α, x, α_max, θ_lim=[0, 0]):
    if α > 0:
        if θ_lim[1] < α_max:
            θ = α
            θ_lim[1] = α
        else:
            θ = α_max
    else:
        if θ_lim[0] > -α_max:
            θ = α
            θ_lim[0] = α
        else:
            θ = -α_max
    return θ_lim[0], θ_lim[1]
    

animator = Animation(threshold_hook=threshold_hook)
α_range = np.concatenate([
    np.linspace(0, 1.8, num=150),
    np.linspace(1.8, -1.8, num=150),
    np.linspace(-1.8, .8, num=150),
]).tolist()

puts = [[[α, 10]] for α in α_range]
x = repeat(10)

html = make_animation(animator, puts, x)
HTML(html)

In [9]:
with open('assets/animations/threshold.html', 'w') as f:
    f.write(html)

# Distance

In [10]:
animator = Animation()
α_range = repeat([.8])
x = np.linspace(10, 5, num=250).tolist()
x += x[::-1]

puts = [[[.8, xi]] for xi in x]

html = make_animation(animator, puts, x)
HTML(html)

In [11]:
with open('assets/animations/distance.html', 'w') as f:
    f.write(html)

# Put Success

In [12]:
animator = Animation()
α_range = sorted([α_min, α_min / 2, -1., 0., -.4, .5, α_max / 2, α_max])

puts = [
    [[αi, xi] for αi in α_range]
    for xi in np.linspace(-1, 10, num=250)
]
for put in puts:
    put.append([.8, 10])

x = repeat(10)

html = make_animation(animator, puts, x)
HTML(html)

In [13]:
with open('assets/animations/simulations.html', 'w') as f:
    f.write(html)

# PDF

In [14]:
animator = Animation(pdf=True)
α_range = sorted([α_min, α_min / 2, -1., 0., -.4, .5, .8, α_max / 2, α_max])
puts_base = list(zip(α_range, repeat(10)))
puts = [
    puts_base + [[α, 10]]
    for α in np.linspace(α_min, α_max, num=250)
]
x = repeat(10)
pdf_range = [None, *np.linspace(α_min, α_max, num=250)[1:]]

html = make_animation(animator, puts, x, pdf_range)
HTML(html)

In [15]:
with open('assets/animations/pdf.html', 'w') as f:
    f.write(html)

# PDF Continuous

In [16]:
animator = Animation(pdf=True)
α_range = sorted([α_min, α_min / 2, -1., 0., -.4, .5, α_max / 2, .8, α_max])
puts_base = list(zip(α_range, repeat(10)))
puts = [
    puts_base + [[α, 10]]
    for α in np.concatenate([
        np.linspace(α_max, α_min, num=250),
        np.linspace(α_min, α_max, num=250),
    ])
]
x = repeat(10)
pdf_range = repeat(α_max)
pdf_marker = np.concatenate([
    np.linspace(α_max, α_min, num=250),
    np.linspace(α_min, α_max, num=250),
])

html = make_animation(animator, puts, x, pdf_range, pdf_marker, repeat=True)
HTML(html)

In [17]:
with open('assets/animations/pdf-continuous.html', 'w') as f:
    f.write(html)